<a class='anchor' id="summary"></a>
### [Table of Contents](#summary)


- **[Day 1](#day1)**
    - [Intro : check the original dataset](#intro)
    - [Part I : Data rows preprocessing](#part1)
    - [Part II : Data set preprocessing](#part2)
    - [Part III : Refactorization](#part3)
- **[Day 2](#day2)**
    - [Summary statistics](#statistics)
    - [Imputing missing values](#missing)
    - [Compute value_counts for categorical columns](#value_counts)
    - [Data lookup](#lookup)
        - [Question 1](#question1)
        - [Question 2](#question2)
        - [Question 3](#question3)
        - [Question 4](#question4)
        - [Question 5](#question5)

<a class='anchor' id="day1"></span>
# <span style="color:blue"> Day 1</span>
[Table of Contents](#summary)

<a class='anchor' id='intro'></a>
## Intro : check the original dataset

You may download the original dataset from :
https://www.kaggle.com/PromptCloudHQ/innerwear-data-from-victorias-secret-and-others
- Explore the meta-data of the dataset
- Understand the global content of the dataset
- Check for the different variables and their type

For this class, we will use a slightly modified version available here :
https://github.com/jaotombo/data_science_from_scratch/tree/main/data
<br>Download and unzip the `victoria` file into this course's data folder

[Table of Contents](#summary)

<a class='anchor' id='part1'></a>
## Part I : Data rows preprocessing
[Table of Contents](#summary)

In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
os.getcwd()

'D:\\database'

In [14]:
DATA_PATH = r'D:\database'

In [15]:
# os.chdir('../../data')
os.chdir(DATA_PATH)

### Check a few data points (steps 1 and 2)

**Check a few lines** (the first 3 or 5 lines - enough to be able to check the content) of the data file `data/victoria.csv`, 

**Pick one line, and see how to process it.**

Which means: Check how lines are structured
- what kind of delimiter
- which variables (column names)
- which type of data (str, float, int, ...)
- Save one line for later analysis in one variable

In [16]:
with open('victoria.csv','r', encoding='utf-8') as file:
    count = 0
    for line in file:
        print(line)
        count += 1
        if count > 3 : break

product_name*mrp*price*pdp_url*brand_name*product_category*retailer*description*rating*review_count*style_attributes*total_sizes*available_size*color

Very Sexy Strappy Lace Thong Panty*$14.50 *$14.50 *https://www.victoriassecret.com/panties/shop-all-panties/strappy-lace-thong-panty-very-sexy?ProductID=328310&CatalogueType=OLS*Victoria's Secret*Strappy Lace Thong Panty*Victoriassecret US*Lots of cheek peek, pretty lace, a strappy back—this sexy panty is so not subtle. Allover lace with front bow V-back with crisscross straps Low rise Minimal back coverage: lots of cheek peek Imported nylon/spandex****"[""XS"", ""S"", ""M"", ""L"", ""XL""]"*S*peach melba

Very Sexy Strappy Lace Thong Panty*$14.50 *$14.50 *https://www.victoriassecret.com/panties/shop-all-panties/strappy-lace-thong-panty-very-sexy?ProductID=328310&CatalogueType=OLS*Victoria's Secret*Strappy Lace Thong Panty*Victoriassecret US*Lots of cheek peek, pretty lace, a strappy back—this sexy panty is so not subtle. Allover lace wi

In [17]:
one_line = line

## Experiment with one line to define data preprocessing

In [18]:
# print the last line read in the dataset
one_line

'Very Sexy Strappy Lace Thong Panty*$14.50 *$14.50 *https://www.victoriassecret.com/panties/shop-all-panties/strappy-lace-thong-panty-very-sexy?ProductID=328310&CatalogueType=OLS*Victoria\'s Secret*Strappy Lace Thong Panty*Victoriassecret US*Lots of cheek peek, pretty lace, a strappy back—this sexy panty is so not subtle. Allover lace with front bow V-back with crisscross straps Low rise Minimal back coverage: lots of cheek peek Imported nylon/spandex****"[""XS"", ""S"", ""M"", ""L"", ""XL""]"*S*plum dust\n'

In [19]:
# print an save the first line of the dataset
with open('victoria.csv','r') as file:
    for line in file:
        print(line)
        break

columns = line

product_name*mrp*price*pdp_url*brand_name*product_category*retailer*description*rating*review_count*style_attributes*total_sizes*available_size*color



In [20]:
# split the first line using * as a delimiter : it outputs a list
columns = columns.split('*')
columns

['product_name',
 'mrp',
 'price',
 'pdp_url',
 'brand_name',
 'product_category',
 'retailer',
 'description',
 'rating',
 'review_count',
 'style_attributes',
 'total_sizes',
 'available_size',
 'color\n']

In [21]:
# strip all the white spaces in the columns variable
columns = [var.strip() for var in columns]
columns

['product_name',
 'mrp',
 'price',
 'pdp_url',
 'brand_name',
 'product_category',
 'retailer',
 'description',
 'rating',
 'review_count',
 'style_attributes',
 'total_sizes',
 'available_size',
 'color']

In [22]:
# split the fourth line of the dataset
line_list = one_line.split('*')
line_list

['Very Sexy Strappy Lace Thong Panty',
 '$14.50 ',
 '$14.50 ',
 'https://www.victoriassecret.com/panties/shop-all-panties/strappy-lace-thong-panty-very-sexy?ProductID=328310&CatalogueType=OLS',
 "Victoria's Secret",
 'Strappy Lace Thong Panty',
 'Victoriassecret US',
 'Lots of cheek peek, pretty lace, a strappy back—this sexy panty is so not subtle. Allover lace with front bow V-back with crisscross straps Low rise Minimal back coverage: lots of cheek peek Imported nylon/spandex',
 '',
 '',
 '',
 '"[""XS"", ""S"", ""M"", ""L"", ""XL""]"',
 'S',
 'plum dust\n']

In [23]:
# a function that strips the white space from an element of a list
def clean_last(element):
    return element.strip()

In [24]:
# apply it on the last element of the fourth row of the dataset
clean_last(line_list[-1])

'plum dust'

In [25]:
# a function that cleans a row of the dataset with a given delimiter
def clean_line(row, delimiter):
    row_list = row.split(delimiter)
    return [val.strip() for val in row_list]

In [26]:
# apply on the saved line, with the relevant delimiter *
clean_line(one_line,'*')

['Very Sexy Strappy Lace Thong Panty',
 '$14.50',
 '$14.50',
 'https://www.victoriassecret.com/panties/shop-all-panties/strappy-lace-thong-panty-very-sexy?ProductID=328310&CatalogueType=OLS',
 "Victoria's Secret",
 'Strappy Lace Thong Panty',
 'Victoriassecret US',
 'Lots of cheek peek, pretty lace, a strappy back—this sexy panty is so not subtle. Allover lace with front bow V-back with crisscross straps Low rise Minimal back coverage: lots of cheek peek Imported nylon/spandex',
 '',
 '',
 '',
 '"[""XS"", ""S"", ""M"", ""L"", ""XL""]"',
 'S',
 'plum dust']

In [27]:
# apply on the first line, with the relevant delimiter *
clean_line(line,'*')

['product_name',
 'mrp',
 'price',
 'pdp_url',
 'brand_name',
 'product_category',
 'retailer',
 'description',
 'rating',
 'review_count',
 'style_attributes',
 'total_sizes',
 'available_size',
 'color']

In [28]:
def conv(price):
    return (float(price.strip('$') if '$' in price else np.nan))

In [29]:
conv(line_list[1])

14.5

In [30]:
# a function that converts price into float, otherwise into NANs
def conv_price(l_value):
    if '$' in l_value :
        value = l_value.strip('$')
    else:
        value = np.nan
    return float(value)

In [31]:
# applying the function on the two price variables
for value in line_list[1:3]:
    print(conv_price(value))

14.5
14.5


In [32]:
# applying the function on the two price variables- version 2
for j in range(1,3):
    print(columns[j], conv_price(line_list[j]))

mrp 14.5
price 14.5


In [1]:
def repl_nan(value):
    if value =='':
        value = np.nan
    else: value = float(value)
    return value

In [34]:
for value in line_list[8:11]:
    print(repl_nan(value))


nan
nan
nan


In [35]:
line_list

['Very Sexy Strappy Lace Thong Panty',
 '$14.50 ',
 '$14.50 ',
 'https://www.victoriassecret.com/panties/shop-all-panties/strappy-lace-thong-panty-very-sexy?ProductID=328310&CatalogueType=OLS',
 "Victoria's Secret",
 'Strappy Lace Thong Panty',
 'Victoriassecret US',
 'Lots of cheek peek, pretty lace, a strappy back—this sexy panty is so not subtle. Allover lace with front bow V-back with crisscross straps Low rise Minimal back coverage: lots of cheek peek Imported nylon/spandex',
 '',
 '',
 '',
 '"[""XS"", ""S"", ""M"", ""L"", ""XL""]"',
 'S',
 'plum dust\n']

In [39]:
def conv_size(string):
    import regex as re
    size_list = string.split(',')
    return [re.sub('[^A-Z0-9]','',size) for size in size_list]

In [40]:
conv_size(line_list[-3])

['XS', 'S', 'M', 'L', 'XL']

In [53]:
def preproc(line_list):
    '''
    replaces all non $ prices (columns 1&2) into nan's
    replaces all empty values (columns 8-10) into nan's
    converts total size values into size list
    removes the '\n' character at the end of each row
    '''
    
    line_list[1:3] = [conv_price(price) for price in line_list[1:3]]
    line_list[8:11] = [repl_nan(value) for value in line_list[8:11]]
    line_list[-3] = conv_size(line_list[-3])
    line_list[-1] = clean_last(line_list[-1])
    return line_list
    

In [54]:
preproc(line_list)

TypeError: argument of type 'float' is not iterable

In [61]:
df_raw = dict()
with open('victoria.csv','r',encoding = 'utf-8')as file:
    idx = -1
    for line in file:
        row = line.split('*')
        if idx == -1:
            df_raw['columns'] = row
            idx+=1
        else:
            df_raw[idx] = row
            idx+=1

In [62]:
len(df_raw)

453387

In [63]:
import copy
raw_df = copy.deepcopy(df_raw)

In [58]:
df_raw['columns']

['product_name',
 'mrp',
 'price',
 'pdp_url',
 'brand_name',
 'product_category',
 'retailer',
 'description',
 'rating',
 'review_count',
 'style_attributes',
 'total_sizes',
 'available_size',
 'color\n']

In [64]:
df_proc = {idx:preproc(row) for idx,row in df_raw.items() if type(idx)==int}

In [65]:
count = 0
for row in df_raw.values():
    if row[1] is np.nan or row[2] is np.nan:
        count +=1
print(count)

506


In [66]:
df_proc_nan = {idx:row for idx,row in df_proc.items() if (row[1] is not np.nan and row[2] is not np.nan)}

In [67]:
len(df_proc_nan)

452880

In [68]:
mrp = [row[1] for row in df_proc_nan.values()]

In [69]:
np.mean(mrp)

36.028994877230176

In [70]:
sum(mrp)/len(mrp)

36.028994877193064

In [71]:
np.median(mrp)

36.0

In [72]:
np.std(mrp,ddof=1) #degree of freedom n-1

14.7628850988695

In [73]:
np.percentile(mrp,25)

29.5

In [75]:
for idx,name in enumerate(raw_df['columns']):
    #enumerate 遍历一个可迭代序列，返回带索引的序列，默认索引从0开始
    print(idx,name)

0 product_name
1 mrp
2 price
3 pdp_url
4 brand_name
5 product_category
6 retailer
7 description
8 rating
9 review_count
10 style_attributes
11 total_sizes
12 available_size
13 color



In [76]:
def num_col(df,col_index):
    col = []
    for row in df.values():
        if type(row[col_index]) not in [float,int]:
            print('No numerical')
            break
        elif type(row[col_index]) in [float,int] and row[col_index] is not np.nan:
            col.append(row[col_index])
    return col

In [77]:
col1 = num_col(df_proc,8)
len(col1)

137734

In [80]:
def extract_num_col(df,col_index):
    return [row[col_index] for row in df.values() if type(row[col_index]) in [float,int] and row[col_index]is not np.nan]

In [82]:
col2 = extract_num_col(df_proc,8)
len(col2)

137734